In [1]:
from cspython.scraper import modifiedSoup, get_teamID
from cspython.data_processing import process_scrapped
import cPickle as pkl
import sys
import pandas as pd
sys.setrecursionlimit(15000)

def get_series(big_data):
    return big_data[big_data.keys()[0]]

In [2]:
with open('NRG_processed.pkl', 'rb') as f:
    ov, s = pkl.load(f)

In [67]:
map_pool = ['Cache', 'Mirage', 'Inferno', 'Nuke', 'Train', 'Cobblestone', 'Overpass']
cur_roster = ['RIKO', 'FugLy', 'Brehze', 'AnJ', 'CeRq', 'daps']  NRG
cur_roster = ['gob b', 'LEGIJA', 'tabseN', 'nex', 'keev', 'kakafu'] Big
cur_roster = ['Skadoodle', 'Stewie2k', 'autimatic', 'RUSH', 'tarik', 'valens'] Cloud9
cur_roster = ['reltuC', 'koosta', 'nahtE', 'FNS', 'Rickeh', 'Ryu'] Counter Logic Gaming
cur_roster = ['chrisj', 'oskar', 'ropz', 'suNny', 'STYKO', 'lmbt'] mousesports
cur_roster = ['AZR', 'jks', 'USTILO', 'Nifty', 'NAF', 'kassad']  Renegades
cur_roster = ['dzt', 'land1n', 'tatazin', 'shz', 's1', 'bLecker'] Tempo Storm
cur_roster = ['ANGE1', 'Zero', 'DeadFox', 'woxic', 'ISSAA', 'Johnta']  HellRaisers

In [23]:
matches = pd.Series(name='num_matches')
for idx, series in s.iteritems():
    
    if series['scoreboards'][0][0].columns[0] == 'NRG':
        r=series['scoreboards'][0][0].loc[:,'NRG'].tolist()
    else:
        r=series['scoreboards'][1][0].loc[:,'NRG'].tolist()
    r = [get_alias(raw_name) for raw_name in r]
    num_matches = count_matches(r, cur_roster)
    matches.loc[idx]=num_matches

In [21]:
def count_matches(r, cur_roster):
    c=0
    for name in r:
        if name in cur_roster:
            c+=1
    return c

In [22]:
def get_alias(raw_name):
    return str(raw_name.split("'")[1])

In [30]:
matches.value_counts()
good_matches = matches.index[matches==5].tolist()

In [33]:
total_matches = []
for idx in good_matches:
    series = s[idx]
    if series['map_pool'] == map_pool:
        total_matches.append(idx)

In [36]:
ov.loc[ov.id == idx, 'url']

24    https://www.hltv.org/matches/2313844/ghost-vs-...
Name: url, dtype: object

In [75]:
m = create_veto_matrix(total_matches, s, map_pool, 'NRG')

In [74]:
def create_veto_matrix(matche_idxs, serieses, map_pool, team_name):
    veto_matrix = pd.DataFrame(index=map_pool, columns=map_pool).fillna(0)
    for idx in matche_idxs:
        vetos= serieses[idx]['vetos']
        remaining = list(map_pool)
        if vetos is None:
            continue
        for v in vetos:
            picked_map = False
            if 'left over' in v:
                continue
            if team_name not in v:
                remaining.remove(v.split(' ')[-1])
            else:
                if 'picked' in v:
                    picked_map = v.split('picked')[-1].strip()
                elif 'removed' in v:
                    removed_map = v.split('removed')[-1].strip()
                if picked_map:
                    remaining.remove(picked_map)
                    for m in remaining:
                        veto_matrix.loc[picked_map, m] -= 1
                else:
                    remaining.remove(removed_map)
                    for m in remaining:
                        veto_matrix.loc[removed_map, m] += 1

    return veto_matrix

In [76]:
m

,Cache,Mirage,Inferno,Nuke,Train,Cobblestone,Overpass
Cache,0,3,2,2,4,1,3
Mirage,-1,0,-2,0,-2,-1,-3
Inferno,1,0,0,-1,0,0,-1
Nuke,4,1,4,0,4,3,3
Train,2,3,2,1,0,2,3
Cobblestone,-3,-1,-2,1,-2,0,-2
Overpass,1,1,3,1,1,1,0


In [78]:
for idx in total_matches:
    vetos = s[idx]['vetos']
    if vetos is None:
        continue
    else:
        for v in vetos:
            print v
        print '######' 

1. Denial removed Mirage
2. NRG removed Overpass
3. Denial picked Cobblestone
4. NRG picked Train
5. Denial removed Cache
6. NRG removed Nuke
7. Inferno was left over
######
1. Tempo Storm removed Mirage
2. NRG removed Nuke
3. Tempo Storm picked Overpass
4. NRG picked Cobblestone
5. Tempo Storm removed Inferno
6. NRG removed Cache
7. Train was left over
######
1. Luminosity removed Mirage
2. NRG removed Nuke
3. Luminosity picked Train
4. NRG picked Cobblestone
5. Luminosity removed Overpass
6. NRG removed Inferno
7. Cache was left over
######
1. Cloud9 removed Nuke
2. NRG removed Train
3. Cloud9 picked Inferno
4. NRG picked Mirage
5. Cloud9 removed Cobblestone
6. NRG removed Cache
7. Overpass was left over
######
1. Misfits removed Cobblestone
2. NRG removed Cache
3. Misfits picked Nuke
4. NRG picked Inferno
5. Misfits removed Train
6. NRG removed Overpass
7. Mirage was left over
######
1. NRG removed Cache
2. Renegades removed Nuke
3. NRG picked Cobblestone
4. Renegades picked Overpas